# 목표 : 
1. article의 경우 10차원으로 표현이 가능하다(articles.csv파일에 product_type: 253 (product_type: vest top)처럼 제품군, 색에 대한 특성 벡터)
따라서 ALS와 달리 customer만 벡터화 하고 상품은 위 특성을 이용하여 고정된 벡터로 표현해볼 수 있다.
2. 가변 customer vector와 정적 article vector를 행렬 곱셈을 하고 실제 구매 데이터 테이블과 비교한다.
* customer vector는 ALS의 경우 만들 수 있지만 지금처럼 article vector를 고정한 상태일 경우 만드는 상황이 아니다. DCGAN처럼 노이즈를 입력을 받는다.
* 문제 손실함수 설정이 어려움(label이 customer에 대한 label이 아니고 article vector가 곱해진 것에 대한 라벨이라 반대방향으로 가중치 수정이 이루어질 수 있음)

# **DOWNLOAD DATA**

In [ ]:
import pandas as pd
import os

fname_tran ='../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
fname_cus ='../input/h-and-m-personalized-fashion-recommendations/customers.csv'
fname_article ='../input/h-and-m-personalized-fashion-recommendations/articles.csv'

In [ ]:
def loadData(filePAth):
    return pd.read_csv(filePAth, sep=',')

data_cus = loadData(fname_cus)
data_article = loadData(fname_article)
data = loadData(fname_tran)

In [ ]:
using_cols = ['article_id', 'product_type_no', 'colour_group_code', 'index_group_no']
data_article_code = data_article[using_cols]

# **PREPROCESS DATA**

In [ ]:
# 고유한 이름과 코드를 인덱스화합니다.
def initial_embedding(DataFrame, id_to_idx, targetColumn):

    temp_data = DataFrame[targetColumn].map(id_to_idx.get).dropna()

    if len(temp_data) == len(DataFrame):  
        print('no-null')
        DataFrame[targetColumn] = temp_data   
    else:
        print('detect null')

In [ ]:
# 총 131개의 다른 값을 갖습니다. 이 벡터에 유독 크게 변동이 될 수 있으므로 벡터를 정규화하였습니다.
# 특히 군집화를 통해 비슷한 article 그룹을 만들때 (0,0,0....)으로부터의 거리를 통해 군집화를 시도해보고자 정규화가 필요하다 생각했습니다.

data_article_code_unique = data_article_code.sort_values(by=['product_type_no'])['product_type_no'].unique()
data_article_code_to_idx = {v:k/131.0 for k,v in enumerate(data_article_code_unique)}

initial_embedding(data_article_code, data_article_code_to_idx, 'product_type_no')

In [ ]:
colour_group_code_unique = data_article_code.sort_values(by=['colour_group_code'])['colour_group_code'].unique()
colour_group_code_unique_to_idx = {v:k/49 for k,v in enumerate(colour_group_code_unique)}

initial_embedding(data_article_code, colour_group_code_unique_to_idx, 'colour_group_code')

In [ ]:
index_group_no_unique = data_article_code.sort_values(by=['index_group_no'])['index_group_no'].unique()
index_group_no_unique_to_idx = {v:k/26.0 for k,v in enumerate(index_group_no_unique)}

initial_embedding(data_article_code, index_group_no_unique_to_idx, 'index_group_no')

In [ ]:
articleCSV_unique = data_article_code.sort_values(by=['article_id'])['article_id'].unique()
article_code_to_idx = {v:k for k,v in enumerate(articleCSV_unique)}

initial_embedding(data_article_code, article_code_to_idx, 'article_id')

In [ ]:
# 데이터프레임 열을 벡터로 바꿈
data_article_code['vector'] = [[data_article_code['product_type_no'][i],\
                                data_article_code['colour_group_code'][i],\
                               data_article_code['index_group_no'][i]] \
                               for i in range(len(data_article_code))]

In [ ]:
article_code_to_vector = {data_article_code['article_id'][i]:data_article_code['vector'][i] for i in range(len(data_article_code)) }

In [ ]:
temp_article_code_data = data['article_id'].map(article_code_to_vector.get).dropna()
print(len(temp_article_code_data),len(data))
if len(temp_article_code_data) == len(data):
    print('no-null')
    data['article_vector'] = temp_article_code_data
else:
    print('detect null')
data

In [ ]:
user_unique = data['customer_id'].unique()
article_unique = data['article_id'].unique()

In [ ]:
user_to_idx = {v:k for k,v in enumerate(user_unique)}
article_to_idx = {v:k for k,v in enumerate(article_unique)}

In [ ]:
initial_embedding(data, user_to_idx, 'customer_id')

data

# MODEL

# 목표:
1. customer를 임베딩하는 레이어를 포함한 모델을 만든다.
-> 행렬 곱을 하려면 article의 벡터 차원에 맞춰야하므로 출력을 1~10사이의 article차원에 맞춘다.

In [ ]:
import tensorflow as tf
import numpy as np
import time
from scipy.sparse import csr_matrix

In [ ]:
USER_COUNT = len(user_unique)
ARTICLE_COUNT = len(article_code_to_idx)
DATA_NUM = data.shape[0]

optimizer = tf.keras.optimizers.Adam(1e-4)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
model_customer = tf.keras.Sequential([
    tf.keras.layers.Embedding(ARTICLE_COUNT, 64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3)
])

In [ ]:
# 정적 데이터인 customer를 모델이 학습을 통해 처리한다기 보단 GAN과 같이 의미없는 데이터를 의미있게 만드는 것이 이 문제의
# 핵심이라고 가정했습니다. 다만 노이즈를 입력으로 받을때 음수처리에 대한 문제가 있고,
# 임베딩한 데이터를 임베팅 레이어(모델의 레이어)에 주입하는 거와 차이가 없는것 같아 아래에서는 사용하지는 않았습니다.
noise = tf.random.normal([DATA_NUM,1])

In [ ]:
# articles.csv파일의 article정보를 (3, article총 개수 )형태의 행렬로 만듭니다.
# customer를 (1,3)의 행렬로 만들어 둘을 곱하면 한 customer가 article들에 대한 구매 가능성이 되고 이를 실제 구매 데이터에 비교하려고
# 시도했습니다.

y_article_input = [np.array(v) for k,v in enumerate(data_article_code['vector'])]
y_article_input = np.array(y_article_input)

def transpose_matrix(matrix):
    return matrix.T
y_article_input = transpose_matrix(y_article_input)

y_article_input.shape

In [ ]:
Matrix = tf.matmul(model_customer(noise[5], training=True), y_article_input)
Matrix

In [ ]:
encoding = article_code_to_idx[data['article_id'][0]]
encoding = tf.one_hot(encoding, ARTICLE_COUNT)
encoding.shape

In [ ]:
def compare_loss( max_value_index, real_value_index ):
    return cross_entropy(real_value_index,max_value_index)

In [ ]:
# 우려되는 점은 articles 행렬을 곱함으로서 기울기의 부호자체가 변하지 않을까 하는 것입니다. 
# 실제로 약간의 테스트성으로 학습을 해본 결과 음으로 증폭되버리고 있다는 것입니다.

@tf.function
def train_step(customer_id, encoding):
    
    with tf.GradientTape() as tape:
        x_user = model_customer(customer_id, training=True)
        Matrix = tf.matmul(x_user, y_article_input)
        loss = compare_loss(Matrix, encoding)

    gradients = tape.gradient(loss, model_customer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model_customer.trainable_variables))

In [ ]:
model_customer.summary()

In [ ]:
# model_customer.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
#               optimizer=tf.keras.optimizers.Adagrad(0.5),
#               metrics=['accuracy'],
#              )

In [ ]:
customers = data['customer_id']
customers = np.array(customers)

In [ ]:
# 모델에 대한 테스트를 위해 article을 3차원 표현으로 줄였고, 학습하는 데이터도 줄였습니다. 1000명의 customer에 대해서만 10번 반복합니다.

limit = 1000
def train(data, epochs):
    for epoch in range(epochs):
        num = 0
        time_count_1 = 0
        starts = time.time()

        for i in range(limit):
            start = time.time()
            encoding = [article_code_to_idx[data['article_id'][i]]]
            encoding = tf.one_hot(encoding, ARTICLE_COUNT)
            train_step(customers[tf.newaxis, i],encoding)
 
            end = time.time()
            time_count_1 = (end - start)
            if num%100 == 0 :
                print('.' , end = ' ')
            if num%300 == 0:
                time_left = (((limit-num) / 1) * time_count_1 / 60)
                print(f"{time_count_1:.5f} sec / TIME_LEFT(min): ",time_left)
                time_count_1 = 0
            num = num +1
        # print (' 에포크 {} 에서 걸린 시간은 {} 초 입니다'.format(epoch +1, time.time()-start))
        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-starts))

In [ ]:
%%time
history = train(data, 10)

# CHECK

In [ ]:
def predict(num):
    tem = tf.matmul(model_customer(customers[tf.newaxis, num], training=False), y_article_input)[0]
    prediction_list = tf.math.argmax(tem)
    return prediction_list,tem

In [ ]:
#첫번쨰 customer가 산 품목
list = data[data['customer_id']==0]['article_id'].to_numpy()
list

In [ ]:
prediction ,predicted_vector = predict(0)
#모든 출력이 음수..
print(prediction, predicted_vector)

In [ ]:
# 38604로 모둔 입력에 대해 수렴하는 문제가 발생합니다.
predicted_vector[38604]

In [ ]:
#아이디값을 다시 article_id로 변환
idx_to_article_code = {v:k for k,v in article_code_to_idx.items()}
idx_to_article_code[38604]

**check most suitable item**

In [ ]:
from IPython.display import Image

In [ ]:
k = [658223002]

num = 0
Image(f'../input/h-and-m-personalized-fashion-recommendations/images/0{str(k[num])[:2]}/0{int(k[num])}.jpg' , width = 200)

In [ ]:
k = list

num = 0 #check what the customer bought 0<= num <=9
Image(f'../input/h-and-m-personalized-fashion-recommendations/images/0{str(k[num])[:2]}/0{int(k[num])}.jpg' , width = 200)

# ANOTHER MODEL

In [ ]:
# TATAL_NUM = int(len(data)/10)

# user_unique = data['customer_id'].unique()
# user_to_idx = {v:k for k,v in enumerate(user_unique)}
# temp_user_data = data['customer_id'].map(user_to_idx.get).dropna()

# if len(temp_user_data) == len(data):  
#     print('no-null')
#     data['customer_id'] = temp_user_data   
# else:
#     print('detect null')
# data

In [ ]:
# train_data = data[:TATAL_NUM]
# ##
# starts = time.time()
# train_label = [data_article_id_to_idx[data['article_id'][i]] for i in range(TATAL_NUM)]
# end = time.time()
# ##
# time_count_1 = (end - starts)
# time_left = time_count_1
# time_left = time_left
# print(f"{time_count_1:.5f} sec / TIME_LEFT(min): ",time_left)
# from sklearn.model_selection import train_test_split
# train_input, val_input, train_label, val_label = \
#     train_test_split(train_data['customer_id'].to_numpy(), train_label, shuffle=True, test_size = 0.1)
# ##
# def get_matrix_factorization_model():
    
#     item_input = tf.keras.layers.Input(shape=[1], name='Item')
#     item_embedding_layer = tf.keras.layers.Embedding(
#       1000,
#       64,
#       name='ItemEmbedding')
    
#     x = tf.keras.layers.Dense(128, activation='relu')(tf.cast(item_embedding_layer(item_input),tf.int64))
#     print(tf.cast(item_embedding_layer(item_input),tf.int64))
#     x1 = tf.keras.layers.Dense(3)(x)
    
#     pred = tf.keras.layers.Dot(
#     (2,1), name='Dot')([x1, tf.expand_dims(y_article_input,0)])
    
#     pred = tf.cast(pred, tf.int64)
#     pred = tf.math.argmax(pred)
#     pred = tf.cast(pred, tf.int64)
    
#     model = tf.keras.Model(inputs=item_input, outputs=pred)

#     return model

# ##
# model_a = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=(105542, )),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(3)
# ])
# ##
# model = get_matrix_factorization_model()

In [ ]:
# model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#               optimizer=tf.keras.optimizers.Adagrad(0.5),
#               metrics=['accuracy'],
#              )
# ##
# model.summary()
# ##
# train_input = np.array(train_input)
# train_label = np.array(train_label)
# val_input = np.array(val_input)
# val_label = np.array(val_label)
# ##
# history = model.fit(train_input, train_label, epochs=1,
#                     validation_data=(val_input, val_label),
#                     steps_per_epoch=100,
#                     validation_steps=30)

In [ ]:
# import matplotlib.pyplot as plt

# def plot_graphs(history, metric):
#     plt.plot(history.history[metric])
#     plt.plot(history.history['val_'+metric], '')
#     plt.xlabel("Epochs")
#     plt.ylabel(metric)
#     plt.legend([metric, 'val_'+metric])

# plt.figure(figsize=(8, 4))
# plt.subplot(1, 2, 1)
# plot_graphs(history, 'accuracy')
# plt.ylim(None, 1)
# plt.subplot(1, 2, 2)
# plot_graphs(history, 'loss')
# plt.ylim(0, None)

# **SUBMISSION**